In [1]:
# Importacion de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

%matplotlib inline

plt.style.use('default') # para graficos matplotlib
plt.rcParams['figure.figsize'] = (10, 8)

sns.set(style="whitegrid") # grid seaborn

# pd.options.display.float_format = '{:20,.0f}'.format # notacion output

# Acceso a set de datos

In [2]:
path = "data/"
df_props_full = pd.read_csv(path + "train_medium_provinces.csv")
df_test = pd.read_csv(path + "test_medium_provinces.csv")

In [3]:
df_props_full.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [4]:
df_test.columns

Index(['Unnamed: 0', 'id', 'titulo', 'descripcion', 'tipodepropiedad',
       'direccion', 'ciudad', 'provincia', 'antiguedad', 'habitaciones',
       'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'lat',
       'lng', 'fecha', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos'],
      dtype='object')

In [5]:
df_test.id.sort_values()

15113        11
1437         30
12600        79
1732         87
1924        106
          ...  
9061     299943
12609    299946
2055     299955
13082    299957
1885     299983
Name: id, Length: 18177, dtype: int64

In [6]:
df_props_full.id.sort_values()

6898          4
5754         10
46105        14
67607        18
57301        19
          ...  
21133    299975
71466    299977
42507    299979
13045    299997
8273     299999
Name: id, Length: 72576, dtype: int64

# Conversión de tipos

In [7]:
df_props_full['fecha'] = pd.to_datetime(df_props_full['fecha'])
df_test['fecha'] = pd.to_datetime(df_test['fecha'])

In [8]:
# Convierto todos los valores 1/0 a uint8
df_props_full['gimnasio'] = df_props_full['gimnasio'].astype('uint8')
df_props_full['usosmultiples'] = df_props_full['usosmultiples'].astype('uint8')
df_props_full['piscina'] = df_props_full['piscina'].astype('uint8')
df_props_full['escuelascercanas'] = df_props_full['escuelascercanas'].astype('uint8')
df_props_full['centroscomercialescercanos'] = df_props_full['centroscomercialescercanos'].astype('uint8')

df_test['gimnasio'] = df_test['gimnasio'].astype('uint8')
df_test['usosmultiples'] = df_test['usosmultiples'].astype('uint8')
df_test['piscina'] = df_test['piscina'].astype('uint8')
df_test['escuelascercanas'] = df_test['escuelascercanas'].astype('uint8')
df_test['centroscomercialescercanos'] = df_test['centroscomercialescercanos'].astype('uint8')

In [9]:
# Convierto los representables en uint8. Utilizo el tipo de pandas UInt8Dtype para evitar conflicto con NaN
df_props_full['antiguedad'] = df_props_full['antiguedad'].astype(pd.UInt8Dtype())
df_props_full['habitaciones'] = df_props_full['habitaciones'].astype(pd.UInt8Dtype())
df_props_full['garages'] = df_props_full['garages'].astype(pd.UInt8Dtype())
df_props_full['banos'] = df_props_full['banos'].astype(pd.UInt8Dtype())

df_test['antiguedad'] = df_test['antiguedad'].astype(pd.UInt8Dtype())
df_test['habitaciones'] = df_test['habitaciones'].astype(pd.UInt8Dtype())
df_test['garages'] = df_test['garages'].astype(pd.UInt8Dtype())
df_test['banos'] = df_test['banos'].astype(pd.UInt8Dtype())

In [10]:
# Convierto los representables en uint16. Utilizo el tipo de pandas UInt16Dtype para evitar conflicto con NaN
df_props_full['metroscubiertos'] = df_props_full['metroscubiertos'].astype(pd.UInt16Dtype())
df_props_full['metrostotales'] = df_props_full['metrostotales'].astype(pd.UInt16Dtype())

df_test['metroscubiertos'] = df_test['metroscubiertos'].astype(pd.UInt16Dtype())
df_test['metrostotales'] = df_test['metrostotales'].astype(pd.UInt16Dtype())

In [11]:
# Convierto los representables en uint32. Utilizo el tipo de pandas UInt32Dtype para evitar conflicto con NaN
df_props_full['id'] = df_props_full['id'].astype(pd.UInt32Dtype())
df_props_full['idzona'] = df_props_full['idzona'].astype(pd.UInt32Dtype())
# df_props_full['Precio_MEX'] = df_props_full['Precio_MEX'].astype(pd.UInt32Dtype())
# df_props_full['Precio_USD'] = df_props_full['Precio_USD'].astype(pd.UInt32Dtype())
df_props_full['precio'] = df_props_full['precio'].astype('float32')

df_test['id'] = df_test['id'].astype(pd.UInt32Dtype())
df_test['idzona'] = df_test['idzona'].astype(pd.UInt32Dtype())

# Habitaciones

Basado en información obtenida del TP1 elimino habitaciones con valores nulos

In [12]:
df_props_full.habitaciones.isna().value_counts()

False    65628
True      6948
Name: habitaciones, dtype: int64

In [13]:
df_props_full.habitaciones.value_counts()

3     39202
2     15367
4      7707
5      1334
1      1263
6       444
7       121
8        87
10       62
9        41
Name: habitaciones, dtype: int64

In [14]:
features = df_props_full
len(features)

72576

In [15]:
pd.pivot_table(df_props_full.groupby(['tipodepropiedad','habitaciones']).size().to_frame(), index='tipodepropiedad',columns=['habitaciones'])

0                                           \
habitaciones                      1        2        3       4       5      6    
tipodepropiedad                                                                 
Apartamento                    423.0   3509.0   3495.0   278.0    18.0   12.0   
Bodega comercial                10.0      5.0      6.0     4.0     3.0    1.0   
Casa                           646.0  10816.0  32739.0  6891.0  1226.0  377.0   
Casa en condominio              64.0    794.0   2655.0   421.0    37.0   10.0   
Casa uso de suelo                4.0     11.0     29.0    25.0     9.0    6.0   
Departamento Compartido          1.0      6.0      3.0     NaN     NaN    1.0   
Duplex                           4.0     29.0     14.0     2.0     1.0    1.0   
Edificio                         1.0      2.0      9.0     6.0     7.0   18.0   
Huerta                           NaN      3.0      1.0     NaN     NaN    NaN   
Inmuebles productivos urbanos    2.0      NaN      5.0     4.0     1.0    1.0   
Local Comercial                 25.0     29.0     23.0    15.0    10.0    7.0   
Local en centro comercial        2.0      3.0      NaN     NaN     NaN    NaN   
Nave industrial                  1.0      1.0      NaN     NaN     NaN    NaN   
Oficina comercial                5.0      5.0      9.0     5.0     4.0    4.0   
Otros                            2.0      3.0      2.0     NaN     NaN    NaN   
Quinta Vacacional               10.0     28.0     38.0    18.0     3.0    1.0   
Rancho                          18.0     17.0     26.0     3.0     5.0    1.0   
Terreno                         27.0     41.0     28.0     8.0     5.0    2.0   
Terreno comercial                3.0      2.0      7.0     1.0     1.0    NaN   
Terreno industrial               NaN      NaN      NaN     NaN     NaN    NaN   
Villa                           15.0     57.0     96.0    23.0     3.0    2.0   

                                                        
habitaciones                      7     8     9     10  
tipodepropiedad                                         
Apartamento                      1.0   3.0   3.0   1.0  
Bodega comercial                 NaN   NaN   NaN   NaN  
Casa                           108.0  58.0  24.0  26.0  
Casa en condominio               2.0   2.0   1.0   1.0  
Casa uso de suelo                2.0   3.0   1.0   3.0  
Departamento Compartido          NaN   NaN   NaN   NaN  
Duplex                           NaN   1.0   NaN   NaN  
Edificio                         3.0   7.0   4.0  15.0  
Huerta                           NaN   NaN   NaN   NaN  
Inmuebles productivos urbanos    NaN   1.0   2.0   9.0  
Local Comercial                  1.0   4.0   3.0   4.0  
Local en centro comercial        NaN   NaN   NaN   NaN  
Nave industrial                  NaN   NaN   NaN   NaN  
Oficina comercial                NaN   5.0   1.0   1.0  
Otros                            NaN   NaN   NaN   NaN  
Quinta Vacacional                2.0   NaN   1.0   NaN  
Rancho                           NaN   NaN   NaN   NaN  
Terreno                          2.0   1.0   NaN   1.0  
Terreno comercial                NaN   NaN   NaN   1.0  
Terreno industrial               NaN   NaN   1.0   NaN  
Villa                            NaN   2.0   NaN   NaN

In [16]:
features = features.dropna(subset=['lat', 'lng', 'habitaciones'])

print(type(features))
len(features)

<class 'pandas.core.frame.DataFrame'>


28878

# Baños

Basado en información obtenida del TP1 elimino baños con valores nulos

In [17]:
df_props_full.banos.isna().value_counts()

False    64397
True      8179
Name: banos, dtype: int64

In [18]:
features.banos.isna().value_counts()

False    27811
True      1067
Name: banos, dtype: int64

In [19]:
features.banos.value_counts()

2    11330
1     7812
3     6311
4     2358
Name: banos, dtype: int64

In [20]:
pd.pivot_table(df_props_full.groupby(['tipodepropiedad','banos']).size().to_frame(), index='tipodepropiedad',columns=['banos'])

0                          
banos                                1        2        3       4
tipodepropiedad                                                 
Apartamento                     2071.0   3822.0   1343.0   395.0
Bodega comercial                  91.0     46.0      7.0     9.0
Casa                           13486.0  20311.0  12553.0  4826.0
Casa en condominio               789.0   1644.0   1058.0   400.0
Casa uso de suelo                 32.0     69.0     28.0    18.0
Departamento Compartido            4.0      5.0      3.0     NaN
Duplex                            42.0      8.0      1.0     NaN
Edificio                          13.0     23.0     30.0    22.0
Huerta                             3.0      2.0      NaN     NaN
Inmuebles productivos urbanos      1.0      6.0      5.0     4.0
Local Comercial                  192.0    170.0     63.0    46.0
Local en centro comercial          4.0      2.0      NaN     1.0
Nave industrial                    4.0      2.0      3.0     1.0
Oficina comercial                 57.0     64.0     28.0    28.0
Otros                              3.0      3.0      2.0     NaN
Quinta Vacacional                 14.0     42.0     29.0    11.0
Rancho                            37.0     23.0      7.0     3.0
Terreno                           72.0     42.0     10.0     3.0
Terreno comercial                 33.0     17.0      5.0     1.0
Terreno industrial                 NaN      3.0      NaN     NaN
Villa                             48.0     63.0     61.0    16.0

In [21]:
features = features.dropna(subset=['banos'])

print(type(features))
len(features)

<class 'pandas.core.frame.DataFrame'>


27811

# Garages

In [22]:
df_props_full.garages.isna().value_counts()

False    59016
True     13560
Name: garages, dtype: int64

In [23]:
features.garages.isna().value_counts()

False    23255
True      4556
Name: garages, dtype: int64

In [24]:
features.garages.value_counts()

2    12949
1     7336
3     1670
0     1300
Name: garages, dtype: int64

In [25]:
pd.pivot_table(df_props_full.groupby(['tipodepropiedad','garages']).size().to_frame(), index='tipodepropiedad',columns=['garages'])

0                          
garages                             0        1        2       3
tipodepropiedad                                                
Apartamento                     481.0   3077.0   3301.0   164.0
Bodega comercial                145.0     11.0     18.0    14.0
Casa                           3683.0  11940.0  25359.0  3625.0
Casa en condominio               88.0    961.0   2056.0   267.0
Casa uso de suelo                33.0     29.0     41.0    15.0
Departamento Compartido           5.0      3.0      3.0     NaN
Duplex                            3.0     36.0     10.0     2.0
Edificio                        119.0     16.0     25.0    21.0
Huerta                           10.0      NaN      NaN     NaN
Inmuebles productivos urbanos    21.0      1.0      6.0     1.0
Local Comercial                 355.0     65.0     94.0    31.0
Local en centro comercial        13.0      2.0      NaN     2.0
Lote                              1.0      NaN      NaN     NaN
Nave industrial                   5.0      NaN      2.0     2.0
Oficina comercial                84.0     40.0     50.0    22.0
Otros                             2.0      3.0      1.0     1.0
Quinta Vacacional                22.0      3.0     17.0     7.0
Rancho                           30.0      7.0      4.0     4.0
Terreno                        2153.0     12.0     17.0     7.0
Terreno comercial               198.0      1.0      2.0     1.0
Villa                             9.0     48.0     80.0    15.0

In [26]:
pd.pivot_table(df_props_full.groupby(['tipodepropiedad','garages']).size().to_frame(), index='tipodepropiedad',columns=['garages'])

0                          
garages                             0        1        2       3
tipodepropiedad                                                
Apartamento                     481.0   3077.0   3301.0   164.0
Bodega comercial                145.0     11.0     18.0    14.0
Casa                           3683.0  11940.0  25359.0  3625.0
Casa en condominio               88.0    961.0   2056.0   267.0
Casa uso de suelo                33.0     29.0     41.0    15.0
Departamento Compartido           5.0      3.0      3.0     NaN
Duplex                            3.0     36.0     10.0     2.0
Edificio                        119.0     16.0     25.0    21.0
Huerta                           10.0      NaN      NaN     NaN
Inmuebles productivos urbanos    21.0      1.0      6.0     1.0
Local Comercial                 355.0     65.0     94.0    31.0
Local en centro comercial        13.0      2.0      NaN     2.0
Lote                              1.0      NaN      NaN     NaN
Nave industrial                   5.0      NaN      2.0     2.0
Oficina comercial                84.0     40.0     50.0    22.0
Otros                             2.0      3.0      1.0     1.0
Quinta Vacacional                22.0      3.0     17.0     7.0
Rancho                           30.0      7.0      4.0     4.0
Terreno                        2153.0     12.0     17.0     7.0
Terreno comercial               198.0      1.0      2.0     1.0
Villa                             9.0     48.0     80.0    15.0

Basado en informacion del TP1 completo con 0 aquellas propiedades que muestren una tendencia a este valor y elimino el resto

In [27]:
aCompletar = ['Edificio','Bodega comercial','Terreno comercial','Local comercial','Terreno','Rancho','Inmuebles productivos urbanos','Nave industrial','Local en centro comercial']
features.loc[(features.tipodepropiedad.isin(aCompletar)), 'garages'] = features.loc[(features.tipodepropiedad.isin(aCompletar)), 'garages'].fillna(0)
df_test.loc[(df_test.tipodepropiedad.isin(aCompletar)), 'garages'] = df_test.loc[(df_test.tipodepropiedad.isin(aCompletar)), 'garages'].fillna(0)


len(features)

27811

In [28]:
features.loc[(features['tipodepropiedad']=='Casa'),:].garages.isna().value_counts()

False    18439
True      3701
Name: garages, dtype: int64

In [29]:
features.loc[(features['tipodepropiedad']=='Casa en condominio'),:].garages.isna().value_counts()

False    1378
True      368
Name: garages, dtype: int64

In [30]:
features.loc[(features['tipodepropiedad']=='Villa'),:].garages.isna().value_counts()

False    61
True     18
Name: garages, dtype: int64

Completo estas tres propiedades con valor 2 que tiene mayor cantidad de publicaciones

In [31]:
aCompletar2 = ['Casa', 'Casa en condominio', 'Villa']
features.loc[(features.tipodepropiedad.isin(aCompletar2)), 'garages'] = features.loc[(features.tipodepropiedad.isin(aCompletar2)), 'garages'].fillna(2)
df_test.loc[(df_test.tipodepropiedad.isin(aCompletar2)), 'garages'] = df_test.loc[(df_test.tipodepropiedad.isin(aCompletar2)), 'garages'].fillna(2)

len(features)

27811

In [32]:
features.garages.isna().value_counts()

False    27404
True       407
Name: garages, dtype: int64

In [33]:
features = features.dropna(subset=['garages'])

len(features)

27404

# Metros

In [34]:
from sklearn.impute import SimpleImputer

In [35]:
imp = SimpleImputer()
features['metroscubiertos'] = imp.fit_transform(features[['metroscubiertos']])
features['metrostotales'] = imp.fit_transform(features[['metrostotales']])

df_test['metroscubiertos'] = imp.fit_transform(df_test[['metroscubiertos']])
df_test['metrostotales'] = imp.fit_transform(df_test[['metrostotales']])

# Antiguedad

In [36]:
features['antiguedad'] = imp.fit_transform(features[['antiguedad']])

df_test['antiguedad'] = imp.fit_transform(df_test[['antiguedad']])

# Tipo de propiedad

In [37]:
features.tipodepropiedad.isna().value_counts()

False    27399
True         5
Name: tipodepropiedad, dtype: int64

In [38]:
# Drop de datos sin tipo
features = features.dropna(subset=['tipodepropiedad'])
len(features)

27399

# Ciudad

In [39]:
def ciudadesByProvinciaMap(df):
    provs_ciudades = {}
    for i in range(0,len(df)):
        prov = df.loc[i,'provincia']
        city = df.loc[i,'ciudad']
        if ((not pd.isna(prov)) and (not pd.isna(city))):
            if (not prov in provs_ciudades):
                provs_ciudades[prov] = []
            if (not city in provs_ciudades[prov]):
                provs_ciudades[prov].append(city)
    return provs_ciudades

In [40]:
def provinciaByIdZonaMap(df):
    id_prov = {}
    for i in range(0,len(df)):
        prov = df.loc[i,'provincia']
        idzona = df.loc[i,'idzona']
        if ((not pd.isna(prov)) and (not pd.isna(idzona))):
            if (not idzona in id_prov):
                id_prov[idzona] = []
            if (not prov in id_prov[idzona]):
                id_prov[idzona].append(prov)
    return id_prov

# Provincia

One hot encoding

In [41]:
one_hot = pd.get_dummies(features.provincia)
features = features.drop('provincia',axis = 1)
features = features.join(one_hot)

one_hot_test = pd.get_dummies(df_test.provincia)
df_test = df_test.drop('provincia',axis = 1)
df_test = df_test.join(one_hot_test)

features = features.reset_index(drop=True)  # Corrijo index con rows eliminadas para que funcione el for

# Fix Test data set

In [42]:
len(df_test)

18177

In [43]:
cols_dropped = ['fecha', 'id', 'idzona']
cols_string = ['titulo', 'descripcion','tipodepropiedad','direccion', 'ciudad']
df_test.drop(cols_dropped + cols_string, axis=1).count()

Unnamed: 0                    18177
antiguedad                    18177
habitaciones                  16470
garages                       17737
banos                         16119
metroscubiertos               18177
metrostotales                 18177
lat                            8071
lng                            8071
gimnasio                      18177
usosmultiples                 18177
piscina                       18177
escuelascercanas              18177
centroscomercialescercanos    18177
Aguascalientes                18177
Baja California Norte         18177
Baja California Sur           18177
Chiapas                       18177
Chihuahua                     18177
Coahuila                      18177
Colima                        18177
Durango                       18177
Guanajuato                    18177
Guerrero                      18177
Hidalgo                       18177
Michoacán                     18177
Morelos                       18177
Nayarit                     

In [44]:
df_test['habitaciones'] = imp.fit_transform(df_test[['habitaciones']])
df_test['garages'] = imp.fit_transform(df_test[['garages']])
df_test['banos'] = imp.fit_transform(df_test[['banos']])

In [45]:
cols_dropped = ['fecha', 'idzona']
cols_string = ['titulo', 'descripcion','tipodepropiedad','direccion', 'ciudad']
df_test.drop(cols_dropped + cols_string, axis=1).count()

Unnamed: 0                    18177
id                            18177
antiguedad                    18177
habitaciones                  18177
garages                       18177
banos                         18177
metroscubiertos               18177
metrostotales                 18177
lat                            8071
lng                            8071
gimnasio                      18177
usosmultiples                 18177
piscina                       18177
escuelascercanas              18177
centroscomercialescercanos    18177
Aguascalientes                18177
Baja California Norte         18177
Baja California Sur           18177
Chiapas                       18177
Chihuahua                     18177
Coahuila                      18177
Colima                        18177
Durango                       18177
Guanajuato                    18177
Guerrero                      18177
Hidalgo                       18177
Michoacán                     18177
Morelos                     

# Features a CSV

In [46]:
cols_dropped = ['lat', 'lng','fecha', 'id', 'idzona']
cols_string = ['titulo', 'descripcion','tipodepropiedad','direccion', 'ciudad']
features = features.drop(cols_dropped + cols_string, axis=1)
cols_dropped = ['lat', 'lng', 'fecha', 'idzona']
df_test = df_test.drop(cols_dropped + cols_string, axis=1)

In [47]:
features.to_csv('data/features_medium_provinces.csv', index=False)
df_test.to_csv('data/test_medium_provinces_modified.csv', index=True)